imports

In [ ]:
import matplotlib.pyplot as plt
import torch
import sys
import os
from sklearn.metrics import f1_score
from tqdm.notebook import tqdm as tqdm
from tqdm.auto import trange

import warnings
warnings.filterwarnings("ignore")

from myhelpers import config_plots, TrialStatistics
from HGNN.train import CNN, dataLoader
from HGNN.train.configParser import ConfigParser, getModelName, getDatasetName
config_plots.global_settings()

parameters

In [ ]:
experimentsPath="/home/elhamod/HGNN/experiments/"
dataPath="/data/BGNN_data"
experimentName="biology_paper_dataset_differentModels_layers"

generate_confusion_matrix = True

cuda=7

Cuda

In [ ]:
# set cuda
if torch.cuda.is_available():
    torch.cuda.set_device(cuda)
    print("using cuda", cuda)

Setup

In [ ]:
config_parser = ConfigParser(experimentsPath, dataPath, experimentName)

experimentPathAndName = os.path.join(experimentsPath, experimentName)

# instantiate trial stat object
results_dir = os.path.join(experimentPathAndName, "results")
ts = TrialStatistics.TrialStatistics(results_dir)
ts_coarse = TrialStatistics.TrialStatistics(results_dir, "coarse")

datasetManager = dataLoader.datasetManager(experimentPathAndName, dataPath)

paramsIterator = config_parser.getExperiments()  
number_of_experiments = sum(1 for e in paramsIterator)  

Show and save trial statistics

In [ ]:

with tqdm(total=number_of_experiments, desc="experiment") as bar:
    for experiment_params in config_parser.getExperiments():
        print(experiment_params)

        # For analyzing experiments, we don't care about augmentation
        datasetManager.updateParams(config_parser.fixPaths({**experiment_params,**{'augmented': False}}))
        train_loader, validation_loader, test_loader = datasetManager.getLoaders()
        fineList = train_loader.dataset.csv_processor.getFineList()
        coarseList = train_loader.dataset.csv_processor.getCoarseList()
        ordered_fineList_indices = sorted(range(len(fineList)), key=lambda k: fineList[k])
        ordered_coarseList_indices = sorted(range(len(coarseList)), key=lambda k: coarseList[k])
        architecture = {
            "fine": len(fineList),
            "coarse" : len(coarseList)
        }


        
        # Loop through n trials
        for i in trange(experiment_params["numOfTrials"], desc="trial"):
            modelName = getModelName(experiment_params, i)
            trialName = os.path.join(experimentPathAndName, modelName)
            
            # Train/Load model
            print(CNN.getModelFile(trialName))
            model = CNN.create_model(architecture, experiment_params)
            if os.path.exists(CNN.getModelFile(trialName)):
                df, epochs, time_elapsed = CNN.loadModel(model, trialName)
                
                # Update trial outcomes for statistics
                predlist, lbllist = CNN.getLoaderPredictions(test_loader, model, experiment_params)
                ts.addTrialPredictions(experiment_params, predlist, lbllist, ordered_fineList_indices)
                micro_f1 = f1_score(lbllist.cpu(), predlist.cpu(), average='macro')

                predlist, lbllist = CNN.getLoaderPredictionProbabilities(test_loader, model, experiment_params)
                topk = CNN.top_k_acc(predlist, lbllist, topk=(3,5))

                predlist, lbllist = CNN.getLoaderPredictions(test_loader, model, experiment_params, 'coarse')
                ts_coarse.addTrialPredictions(experiment_params, predlist, lbllist, ordered_coarseList_indices)
                micro_f1_coarse = f1_score(lbllist.cpu(), predlist.cpu(), average='macro')

                predlist, lbllist = CNN.getLoaderPredictions(validation_loader, model, experiment_params)
                macro_f1_val = f1_score(lbllist.cpu(), predlist.cpu(), average='macro')

                score = {'loss': CNN.getCrossEntropyFromLoader(test_loader, model, experiment_params),
                         'average correct guess prob': CNN.getAvgProbCorrectGuessFromLoader(test_loader, model, experiment_params),
                         'macro f1 test fine': micro_f1,
                         'macro f1 test coarse': micro_f1_coarse,
                         'macro f1 validation fine': macro_f1_val,
                         'time': time_elapsed,
                         'epochs': epochs,
                         'top-3': topk[0].cpu().numpy(),
                         'top-5': topk[1].cpu().numpy(),
                        }

                ts.addTrial(experiment_params,
                    score, i)
            else:
                print("Model {0} not found!".format(trialName))
        
        bar.update()
        
# Save experiment results
ts.saveStatistics()
ts.saveStatistics(False)
ts.showStatistics()
ts.showStatistics(False)

Show and save confusion matrix

In [ ]:
if generate_confusion_matrix:
    with tqdm(total=number_of_experiments, desc="experiment") as bar:
        for experiment_params in config_parser.getExperiments():
            print(experiment_params)

            ts.printTrialConfusionMatrix(experiment_params, ordered_fineList , printOutput=True)
            ts_coarse.printTrialConfusionMatrix(experiment_params,  ordered_coarseList, printOutput=True)

            bar.update()

Print scatter plot

In [ ]:
ts.trialScatter('macro f1 validation fine', 'macro f1 test fine', 'link_layer', True)

In [ ]:
# ts.trialScatter('macro f1 test fine', 'time', 'learning_rate', True)

In [ ]:
# ts.df = ts.df.rename(columns={"link_layer": "linkLayer"})
# ts.df.boxplot(column=['time'], by=['linkLayer'], figsize=(16, 7))